# Try out the Speech Recognition system

In [1]:
# Third party
import speech_recognition as sr

In [11]:
r = sr.Recognizer()
with sr.Microphone() as source:                # use the default microphone as the audio source
    audio = r.listen(source)                   # listen for the first phrase and extract it into audio data

try:
    print("You said " + r.recognize(audio))    # recognize speech using Google Speech Recognition
except LookupError:                            # speech is unintelligible
    print("Could not understand audio")

You said I want to say something


# Download a video frow vimeo in mp4 format

In [3]:
video_id = "66555533"

In [4]:
import re
import json
import io

# Third party
import sh
import requests

In [5]:
def get_vimeo_file(video_id):
    response = requests.get("http://player.vimeo.com/video/{}".format(video_id))
    json_in_js = re.findall(
        re.compile('''var t=(.+);if\(\!t\.request\)'''), response.content.decode("utf-8")
    )[0]
    video_url = json.loads(json_in_js)["request"]["files"]["h264"]["mobile"]["url"]
    mp4 = requests.get(video_url)
    video_file = io.BytesIO(mp4.content)
    video_file.seek(0)
    return video_file

In [6]:
video = get_vimeo_file(video_id)

# Cut the video into 10 seconds segments WAV files, and try recognizing them

In [20]:
wave_file = io.BytesIO()
video.seek(0)
try:
    sh.ffmpeg("-i", "pipe:", "-f", "wav", "-ac", "1", "-", _in=video, _out=wave_file)
except sh.ErrorReturnCode as e:
    print(e.stderr.decode())
wave_file.seek(0)
r = sr.Recognizer()

with sr.WavFile(wave_file) as source:
    length = 5
    for start in range(0, 60, length):
        print(start)
        try:
            print(r.recognize(r.record(source, duration=5, offset=start)))
        except LookupError:
            print("Not recognized")

0
my name is Anthony
5
Dishonored, GameStop been around
10
what are the ingredients in Tempe AZ
15
100 e 9th
20
2009 in 2010 I was going to say
25
North Hudson community policing same Nissan Titan
30
the 1992 Chicago Illinois
35
and that's easy to kill a hundred Yen film site 115
40
contacts on my Samsung TV
45
Not recognized
50
Not recognized
55
Not recognized


:(